In [1]:
from ortools.linear_solver import pywraplp
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import pandas as pd
import math

In [2]:
# Load data from the Excel file
file_path = "../data/VOSimu-InputInformation-Min.xlsx"

locations_df = pd.read_excel(file_path, sheet_name="Locations")
vehicles_df = pd.read_excel(file_path, sheet_name="Vehicles")
container_orders_df = pd.read_excel(file_path, sheet_name="ContainerOrders")

In [3]:
def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Trim all object columns to ensure consistency across tables
    """
    object_cols = df.select_dtypes('object').columns
    df[object_cols] = df[object_cols].apply(lambda x: x.str.strip())
    datetime_cols = df.select_dtypes('datetime').columns
    df[datetime_cols] = df[datetime_cols].astype('int64') // 10**9
    return df


vehicles_df = preprocess_data(vehicles_df)
locations_df = preprocess_data(locations_df)
container_orders_df = preprocess_data(container_orders_df)

In [4]:
locations_df

,Location Name,X-Coordinate [mm],Y-Coordinate [mm],Capacity limitation (# SC)
0,WS001.01,479241,919607,2.0
1,YARD001.09,212302,950696,NaN
2,YARD001.10,222302,950696,NaN
3,YARD001.11,232302,950696,NaN
4,YARD001.12,242302,950696,NaN
5,YARD001.13,252302,950696,NaN
6,QC003,342320,993371,2.0
7,QC005,541820,993371,2.0
8,QC006,642264,993371,2.0


In [5]:
vehicles_df

,ID,StartLocation,LogOn,LogOff
0,SC001,YARD001.09,1724222400,1724257800
1,SC002,YARD001.10,1724222400,1724257800
2,SC003,YARD001.11,1724222400,1724257800
3,SC004,YARD001.12,1724222400,1724257800
4,SC005,YARD001.13,1724222400,1724257800


In [6]:
container_orders_df

,TractorOrderId,ContainerOrderId,ContainerName,Length,OriginLocation,DestinationLocation,Time first known
0,TO_CO_TFTU000067,CO_TFTU000067,TFTU000067,20,QC005,WS001.01,1724224320
1,TO_CO_TFTU000121,CO_TFTU000121,TFTU000121,40,QC003,WS001.01,1724224860
2,TO_CO_TFTU000156,CO_TFTU000156,TFTU000156,40,QC006,WS001.01,1724225280
3,TO_CO_TFTU000175,CO_TFTU000175,TFTU000175,40,QC003,WS001.01,1724225700
4,TO_CO_TFTU000211,CO_TFTU000211,TFTU000211,40,QC005,WS001.01,1724226300
5,TO_CO_TFTU000235,CO_TFTU000235,TFTU000235,40,QC005,WS001.01,1724226780
6,TO_CO_TFTU000239,CO_TFTU000239,TFTU000239,20,QC005,WS001.01,1724226900
7,TO_CO_TFTU000253,CO_TFTU000253,TFTU000253,40,QC005,WS001.01,1724227260


In [7]:
min_time = vehicles_df["LogOn"].min()
max_time = vehicles_df["LogOff"].max()

In [8]:
# Helper function to calculate Manhattan distance
def manhattan_distance(x1, y1, x2, y2):
    return abs(x1 - x2) + abs(y1 - y2)

In [ ]:
# Convert coordinates to meters since they are given in millimeters
locations_df['X-Coordinate [m]'] = locations_df['X-Coordinate [mm]'] / 1000
locations_df['Y-Coordinate [m]'] = locations_df['Y-Coordinate [mm]'] / 1000

# Function to calculate Euclidean distance between two points
def calculate_distance(x1, y1, x2, y2):
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

# Prepare distance matrix
def prepare_distance_matrix(orders, locations):
    location_dict = {row['Location Name']: (row['X-Coordinate [m]'], row['Y-Coordinate [m]']) for _, row in locations.iterrows()}
    
    # Create a list of all unique locations involved in the orders
    all_locations = set(orders['OriginLocation']).union(set(orders['DestinationLocation']))
    all_locations = list(all_locations)
    
    # Add vehicle start locations
    for _, row in vehicles_df.iterrows():
        all_locations.append(row['StartLocation'])
    
    # Build index mapping for faster lookup
    location_index_map = {location: idx for idx, location in enumerate(all_locations)}
    
    num_locations = len(all_locations)
    distance_matrix = [[0] * num_locations for _ in range(num_locations)]
    
    for i in range(num_locations):
        loc_i = all_locations[i]
        xi, yi = location_dict.get(loc_i, (0, 0))  # Default to origin if not found
        
        for j in range(num_locations):
            loc_j = all_locations[j]
            xj, yj = location_dict.get(loc_j, (0, 0))
            
            distance_matrix[i][j] = int(calculate_distance(xi, yi, xj, yj) * 1000)  # Distance in mm
    
    return distance_matrix, location_index_map

distance_matrix, location_index_map = prepare_distance_matrix(container_orders_df, locations_df)

# Prepare data for OR-Tools
def create_data_model(distance_matrix, container_orders_df, vehicles_df, location_index_map):
    data = {}
    data['distance_matrix'] = distance_matrix
    
    # Each order represents a pickup and delivery pair
    pickups_deliveries = []
    for _, row in container_orders_df.iterrows():
        pickup_index = location_index_map[row['OriginLocation']]
        delivery_index = location_index_map[row['DestinationLocation']]
        pickups_deliveries.append((pickup_index, delivery_index))
    
    data['pickups_deliveries'] = pickups_deliveries
    
    # Vehicles start locations
    starts = []
    for _, row in vehicles_df.iterrows():
        starts.append(location_index_map[row['StartLocation']])
    
    data['starts'] = starts
    data['ends'] = [0] * len(starts)  # All vehicles return to depot or initial position
    
    return data

data = create_data_model(distance_matrix, container_orders_df, vehicles_df, location_index_map)

# Define the routing model
def solve_vrp(data):
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), len(data['starts']), data['starts'], data['ends'])
    routing = pywrapcp.RoutingModel(manager)
    
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    # Add Pickup and Delivery constraints
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index))
        routing.solver().Add(routing.GetDimensionOrDie('Time').CumulVar(pickup_index) <= routing.GetDimensionOrDie('Time').CumulVar(delivery_index))
    
    # Adding time dimension for capacity constraints
    time_per_stop = 60  # 60 seconds per stop
    time_windows = {manager.NodeToIndex(location_index_map[start]): (0, 3600) for start in data['starts']}  # Assuming 4 hours max working time
    
    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        0,  # Slack time (allow waiting time)
        3600,  # Maximum time per vehicle (4 hours in seconds)
        False,  # Force start cumulative to zero
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    
    for location_idx, time_window in time_windows.items():
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    
    # Adding capacity constraints
    capacity = 'Capacity'
    routing.AddDimension(
        0,  # Null slack
        0,  # One truck per location
        2,  # Maximum capacity per vehicle
        True,  # Start cumul to zero
        capacity)
    capacity_dimension = routing.GetDimensionOrDie(capacity)
    
    # Apply capacity constraints at locations with limitations
    for _, row in locations_df.iterrows():
        if pd.notna(row['Capacity limitation (# SC)']):
            index = manager.NodeToIndex(location_index_map[row['Location Name']])
            capacity_dimension.CumulVar(index).SetRange(0, int(row['Capacity limitation (# SC)']))
    
    # Minimize waiting time and maximize throughput
    def empty_travel_time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node] if from_node != to_node else 0
    
    empty_travel_time_callback_index = routing.RegisterTransitCallback(empty_travel_time_callback)
    
    # Set the cost of each arc to the distance plus a penalty for empty travel
    def arc_cost_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        distance = data['distance_matrix'][from_node][to_node]
        empty_travel_penalty = 1000  # Penalty for empty travel
        return distance + (empty_travel_penalty if from_node == to_node else 0)
    
    arc_cost_callback_index = routing.RegisterTransitCallback(arc_cost_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(arc_cost_callback_index)
    
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)
    
    solution = routing.SolveWithParameters(search_parameters)
    
    if solution:
        print("Solution found.")
    else:
        print("No solution found.")
    
    return routing, manager, solution

routing, manager, solution = solve_vrp(data)

# Print the solution
def print_solution(data, manager, routing, solution):
    max_route_distance = 0
    for vehicle_id in range(len(data['starts'])):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance/1000)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance/1000))

if solution:
    print_solution(data, manager, routing, solution)